Name: Christian Hellum Bye

## Predicting a Pulsar Star

### 1. Problem Statement

This is a classification problem where the goal is to predict if some radio signal is due to a pulsar star or Radio Frequency Interference (RFI), i.e. noise. Each signal has eight features and one class. The features are: "Mean of the integrated profile", "Standard deviation of the integrated profile", "Excess kurtosis of the integrated profile", "Skewness of the integrated profile", "Mean of the DM-SNR curve", "Standard deviation of the DM-SNR curve", "Excess kurtosis of the DM-SNR curve" and "Skewness of the DM-SNR curve". The classes are 0 (RFI) and 1 (pulsar star).

### 2. Data Preprocessing

The data is taken from https://www.kaggle.com/pavanraj159/predicting-a-pulsar-star. No changes will be made to the original data set as it is already clean and prepared for analysis. The dataset contains 17 898 total samples of which 1639 are positive (pulsar stars) and 16 259 are negative (RFI). Each feature is a decimal number where the number of decimal points in any given feature is consistent between different samples. Moreover, there are no missing datapoints.

### 3. Machine Learning Model

No model was chosen in deliverable 1, but based on the classification methods discussed in lecture, we will try using Support Vector Machine (SVM). Given only eight features, it seems unnecessary to perform Principal Component Analysis (PCA) or reduce the amnount of data in other ways. Moreover, the Naive Bayes assumption might not apply in this case as we do not expect the features to be mutually independent. The k-NN method was considered and might be used, but it is unsure how much the curse of dimensionality will apply. Thus, SVM was chosen (for now, it might change as we develop more tools).

The dataset will be split like the following: 60 % training, 20 % validation and 20 % test. Using a Linear Suppost Vector Classification Method, we would like to determine the following three hyperparameters: the penalty norm (L1 or L2 regularization), the regularization parameter (the coefficient C that sets the amount of regularization, i.e. determines if the margin is soft or hard) and the loss function - whether to use hinge loss or squared hinge loss. Given three hyperparameters, we have to make sure to have a large enough validation set in order to determine all three. This motivates the splitting ratio. 

A first guess for the hyperparamters would be:

- Using L2 regularization. This is because L1 penalty will make the coefficients of the less important feautures go to 0, effecitively reducing the number of features. With only eigth feautures, this is probably neiher necessary nor desirable as we expect all features to be relevant a priori.

- Setting the regularization parameter to a relatively large number. As an initial test, we would like to check the overlap between the class distributions. A quick way to test this is by making the regularization parameter large, hence making a hard margin. If the accuracy on the validation set is close to that of the training set - i.e. low variance - in this limit, there is little overlap between class distributions.

- Use squared hinge loss function. This penalizes large errors more than the hinge loss function, making the decision boundary to finer. This serves the same purpose as chosing a large regularization parameter.

Of course, these hyperparameters needs to be tested for with the validation set, but the choice listed above represent an initial hypothesis.

### 4. Preliminary results

In [36]:
from sklearn.svm import LinearSVC #the training algorithm
from sklearn.model_selection import train_test_split #to split the dataset
from sklearn.metrics import confusion_matrix, f1_score

In [4]:
import numpy as np

In [8]:
data = np.loadtxt('pulsar_stars.csv', delimiter=',', skiprows=1)

In [14]:
X = data[:, 0:8] #features
y = data[:, 8] #classes

In [17]:
#split the dataset into two parts, 80 % containing training and validation sets, 20 % to the test set
X_train_validation, X_test, y_train_validation, y_test = train_test_split(X, y, test_size=0.2)

#split the larger part of the dataset to two parts: 75 % (= 60 % of the total data) to training set, 25 % (= 20 % of the total)
#to the validation set
X_train, X_validation, y_train, y_validation = train_test_split(X_train_validation, y_train_validation, test_size=0.25)

In [20]:
#sets the hyperparamters as explained in section 3, with class_weight='balanced' to adjust the weights of each class to be
#inversely proportional to the class frequency
svm_clf = LinearSVC(penalty='l2', loss='squared_hinge', C=100, class_weight='balanced')

In [25]:
svm_clf.fit(X_train, y_train) #fits the model to the training data

LinearSVC(C=100, class_weight='balanced', dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

There is a large imbalance between postive and negative samples, so accuracy is not the best way to evaluate our model. Instead, we will use the F1-score, which works well to evaluate the balance between precision (how many of the precited positives are true positives) and recall (how many of the true positives are predicted to be positive). Moreover, F1-score takes into account that there is an uneven distribution between number of positive and negative samples. The formula for F1 is:

$$ F_1 = 2 \frac{Precision \times Recall}{Precision + Recall}, $$

where recall and precision are defined like this:

$$ Recall = \frac{True Positives}{True Positives + False Negatives} $$
$$ Precision = \frac{True Positives}{True Positives + False Positives} $$

In [32]:
#predict the class labels for the training set and validation set
train_predict = svm_clf.predict(X_train)
validation_predict = svm_clf.predict(X_validation)

In [37]:
#compute the f1-scores for the training and validation sets
f1_train = f1_score(y_train, train_predict)
f1_validation = f1_score(y_validation, validation_predict)

In [39]:
print('The f1-scores are: \n')
print('Training set:', f1_train)
print('Validation set:', f1_validation)

The f1-scores are: 

Training set: 0.8785145888594165
Validation set: 0.8892561983471075


We see from the f1-scores that the validation set fits the data better than the training set so we are at least not overfitting. A first trials, the hyperparameters seem reasonable.

We will compute the confusion matrix for the test data:

In [40]:
test_predict = svm_clf.predict(X_test) #the predicted classes for the test data
confusion_test = confusion_matrix(y_test, test_predict)

In [43]:
print('Confusion matrix: \n')
print(confusion_test)

tn, fp, fn, tp = confusion_test.ravel()

print('\nTrue positives', tp)
print('True negatives', tn)
print('False positives', fp)
print('False negatives', fn)

Confusion matrix: 

[[3197   32]
 [  59  292]]

True positives 292
True negatives 3197
False positives 32
False negatives 59


The F1-score for the test set is:

In [44]:
print('F1-score for test set:', f1_score(y_test, test_predict))

F1-score for test set: 0.8651851851851853


We see that the F1-scores for the three parts of the datasets are very similar. As a first test, these F1-scores are acceptable. The greatest value possible for F1 is 1 and it seems feasible to push these results closer to the the theoretical upper bound.

### 5. Next steps

Going forward, we would like to tune the hyperparameters to see if its possible to get a better F1-score. If that is resultless, we might also want to try different models, for example k-NN or naive Bayes just to see how they perform in comparison to SVM. It should be fairly straight forward to implement other algoithms and try out different models, as the data is clean and easy to handle.